In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.insert(1, '/home/r7user5/Desktop/STAT')
import comonotonic as cm
import os
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
import copy
import utils
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import SMOTENC

In [2]:
df = pd.read_csv("Financial Distress.csv")
df['Financial Distress'].values[df['Financial Distress'].values > -0.5] = 0
df['Financial Distress'].values[df['Financial Distress'].values <= -0.5] = 1
df['x80'] = df['x80']-1
df['Financial Distress'] = df['Financial Distress'].astype(int)
distress = df['Financial Distress']
df = df.drop(columns=['Company','Time','Financial Distress'])
df['Financial Distress'] = distress
colnames = [('X'+str(i)) for i in range(df.shape[1]-1)]
colnames.append('Y')
df.columns = colnames

In [3]:
df_train, df_test = train_test_split(df, test_size=0.3, stratify=df[['Y']])
categorical = [79]
cont_col = [i for i in range(79)]+[i for i in range(80,83)]
discrete_feature_val = {79:37}

In [4]:
scaler = preprocessing.StandardScaler()
scale_col_name = ["X"+str(i) for i in cont_col]
df_train[scale_col_name] = scaler.fit_transform(df_train[scale_col_name])
reduced_df_train = utils.outlier_removal(df_train, cont_col)
df_test[scale_col_name] = scaler.transform(df_test[scale_col_name])

/home/r7user5/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/r7user5/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
/home/r7user5/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

In [5]:
x_train = reduced_df_train.iloc[:,:-1].to_numpy()
y_train = reduced_df_train.iloc[:,-1].to_numpy()
smote_nc = SMOTENC(categorical_features=[79], sampling_strategy = "all", random_state=0)
x_train_synthetic, y_train_synthetic = smote_nc.fit_resample(x_train, y_train)
x_test = df_test.iloc[:,:-1].to_numpy()
y_test = df_test.iloc[:,-1].to_numpy()

In [6]:
low_x_train = x_train_synthetic[:,[67,74,75]]
low_x_test = x_test[:,[67,74,75]]

In [7]:
categorical = []
cont_col = [0,1,2]
discrete_feature_val = None
allocation_book = {0:50,1:50,2:50}

In [8]:
ciber = cm.clustered_comonotonic(low_x_train,y_train_synthetic,
                                 discrete_feature_val,cont_col,categorical, 
                                 0, None, corrtype='spearman',
                                 discrete_method='custom', allocation_book = allocation_book)
ciber.run()
ciber_predict = ciber.predict(x_test)
print(classification_report(y_test,ciber_predict))

              precision    recall  f1-score   support

           0       0.94      0.16      0.27      1061
           1       0.03      0.73      0.06        41

    accuracy                           0.18      1102
   macro avg       0.49      0.44      0.17      1102
weighted avg       0.90      0.18      0.26      1102



In [9]:
nb = cm.clustered_comonotonic(low_x_train,y_train_synthetic,
                              discrete_feature_val,cont_col,categorical, 
                              1, None, corrtype='spearman',
                              discrete_method='custom', allocation_book = allocation_book)
nb.run()
nb_predict = nb.predict(x_test)
print(classification_report(y_test,nb_predict))

              precision    recall  f1-score   support

           0       0.94      0.16      0.27      1061
           1       0.03      0.73      0.06        41

    accuracy                           0.18      1102
   macro avg       0.48      0.44      0.16      1102
weighted avg       0.90      0.18      0.26      1102



In [11]:
ciber.print_cluster()

[[0, 1, 2]]
